On Chain

In [465]:
import pandas  as pd

In [466]:
df = pd.read_csv("C:/Python/Template/Data Science/DATATHON_2025/data/on_chain/btc.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Columns: 148 entries, time to principal_market_usd
dtypes: float64(146), object(2)
memory usage: 6.8+ MB


C:\Users\Elzandi Irfan Zikra\AppData\Local\Temp\ipykernel_22676\931115122.py:1: DtypeWarning: Columns (147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Python/Template/Data Science/DATATHON_2025/data/on_chain/btc.csv")


,time,AdrActCnt,AdrBal1in100KCnt,AdrBal1in100MCnt,AdrBal1in10BCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1BCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,...,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d,principal_market_price_usd,principal_market_usd
0,2009-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [467]:
import pandas as pd

# 1) Load & parse
df['time'] = pd.to_datetime(df['time'])
df = df.drop(columns=['principal_market_price_usd', 'principal_market_usd'])
df = df.set_index('time').sort_index()

# 2) Floor timestamps to the hour and dedupe
df.index = df.index.floor('H')
df = df[~df.index.duplicated(keep='last')]

# 3) Build full hourly index for 17 Aug 2017 (00:00–23:00), then later filter
day_start  = pd.Timestamp("2017-08-17 00:00:00")
day_end    = day_start + pd.Timedelta(hours=23)
daily_idx  = pd.date_range(start=day_start, end=day_end, freq="1H")

# 4) Reindex & forward‐fill to cover that full day
df_day = df.reindex(daily_idx).ffill()

# 5) Now drop hours 00:00–03:00, keep from 04:00 onward
df_day = df_day[df_day.index >= pd.Timestamp("2017-08-17 04:00:00")]

# 6) If you need to extend beyond that day, append the rest of your hourly series:
#    build overall index from 04:00 17 Aug to last hour:
overall_idx = pd.date_range(
    start=pd.Timestamp("2017-08-17 04:00:00"),
    end=df.index.max().ceil("H"),
    freq="1H"
)
df_hourly = df.reindex(overall_idx).ffill()

# 7) Combine the day‐one slice with the rest
df_combined = pd.concat([df_day, df_hourly[df_hourly.index > day_end]])

# 8) Reset and save
out = df_combined.reset_index().rename(columns={'index':'timestamp'})
out.to_csv(
    "C:/Python/Template/Data Science/DATATHON_2025/data/on_chain/on_chain_btc.csv",
    index=False
)


C:\Users\Elzandi Irfan Zikra\AppData\Local\Temp\ipykernel_22676\665544169.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df.index = df.index.floor('H')
C:\Users\Elzandi Irfan Zikra\AppData\Local\Temp\ipykernel_22676\665544169.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  daily_idx  = pd.date_range(start=day_start, end=day_end, freq="1H")
C:\Users\Elzandi Irfan Zikra\AppData\Local\Temp\ipykernel_22676\665544169.py:27: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  end=df.index.max().ceil("H"),
C:\Users\Elzandi Irfan Zikra\AppData\Local\Temp\ipykernel_22676\665544169.py:25: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  overall_idx = pd.date_range(


In [468]:
df = pd.read_csv("C:/Python/Template/Data Science/DATATHON_2025/data/on_chain/on_chain_btc.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68421 entries, 0 to 68420
Columns: 146 entries, timestamp to VtyDayRet30d
dtypes: float64(145), object(1)
memory usage: 76.2+ MB


,timestamp,AdrActCnt,AdrBal1in100KCnt,AdrBal1in100MCnt,AdrBal1in10BCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1BCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,...,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d
0,2017-08-17 04:00:00,863099.0,9648.0,1455565.0,6811294.0,928.0,415719.0,3394636.0,65.0,109740.0,...,797542.0,688159.703732,2.964948e+09,2.810758,12110.198487,0.016232,69.934122,36.545227,0.0463,0.058414
1,2017-08-17 05:00:00,863099.0,9648.0,1455565.0,6811294.0,928.0,415719.0,3394636.0,65.0,109740.0,...,797542.0,688159.703732,2.964948e+09,2.810758,12110.198487,0.016232,69.934122,36.545227,0.0463,0.058414
2,2017-08-17 06:00:00,863099.0,9648.0,1455565.0,6811294.0,928.0,415719.0,3394636.0,65.0,109740.0,...,797542.0,688159.703732,2.964948e+09,2.810758,12110.198487,0.016232,69.934122,36.545227,0.0463,0.058414
3,2017-08-17 07:00:00,863099.0,9648.0,1455565.0,6811294.0,928.0,415719.0,3394636.0,65.0,109740.0,...,797542.0,688159.703732,2.964948e+09,2.810758,12110.198487,0.016232,69.934122,36.545227,0.0463,0.058414
4,2017-08-17 08:00:00,863099.0,9648.0,1455565.0,6811294.0,928.0,415719.0,3394636.0,65.0,109740.0,...,797542.0,688159.703732,2.964948e+09,2.810758,12110.198487,0.016232,69.934122,36.545227,0.0463,0.058414


In [1]:
import pandas as pd

# 1. Definisikan path input & output
input_path = r"C:/Python/Template/Data Science/DATATHON_2025/data/on_chain/on_chain_btc.csv"
output_path = r"C:/Python/Template/Data Science/DATATHON_2025/data/on_chain/on_chain_btc2.csv"

# 2. Baca data
df = pd.read_csv(input_path)

# 3. Daftar fitur yang dipilih
selected_features = [
    "timestamp",         # cap waktu
    "AdrActCnt",         # active addresses
    "TxCnt",             # transaction count
    "FeeMeanUSD",        # rata-rata fee (USD)
    "NVTAdj",            # adjusted NVT ratio
    "CapMVRVCur",        # market-to-realized value ratio
    "HashRate",          # hashrate jaringan
    "SplyAct1d",         # active supply 1 hari
    "AdrBalUSD1KCnt",    # count alamat dengan balance ~1k USD
    "SplyAdrTop1Pct",    # supply dipegang top 1%
    "ROI30d"             # return 30-hari
]

# 4. Pastikan semua kolom ada di DataFrame
missing = [c for c in selected_features if c not in df.columns]
if missing:
    raise ValueError(f"Kabutuhan kolom berikut tidak ditemukan di input CSV: {missing}")

# 5. Seleksi kolom
df_sel = df[selected_features]

# 6. Simpan hasil ke CSV baru tanpa index
df_sel.to_csv(output_path, index=False)

print(f"✅ File with selected features written to:\n  {output_path}")


✅ File with selected features written to:
  C:/Python/Template/Data Science/DATATHON_2025/data/on_chain/on_chain_btc2.csv


Off Chain

In [469]:
df = pd.read_csv("C:/Python/Template/Data Science/DATATHON_2025/data/off_chain/ohlcv.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68324 entries, 0 to 68323
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   timestamp            68324 non-null  object 
 1   open                 68324 non-null  float64
 2   high                 68324 non-null  float64
 3   low                  68324 non-null  float64
 4   close                68324 non-null  float64
 5   volume               68324 non-null  float64
 6   close_time           68324 non-null  object 
 7   quote_asset_volume   68324 non-null  float64
 8   num_trades           68324 non-null  int64  
 9   taker_buy_base_vol   68324 non-null  float64
 10  taker_buy_quote_vol  68324 non-null  float64
 11  ignore               68324 non-null  int64  
dtypes: float64(8), int64(2), object(2)
memory usage: 6.3+ MB


,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,taker_buy_base_vol,taker_buy_quote_vol,ignore
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,2017-08-17 04:59:59.999,202366.138393,171,35.160503,150952.477943,0
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,2017-08-17 05:59:59.999,100304.823567,102,21.448071,92608.279728,0
2,2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,2017-08-17 06:59:59.999,31282.312670,36,4.802861,20795.317224,0
3,2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,2017-08-17 07:59:59.999,19241.058300,25,2.602292,11291.347015,0
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,2017-08-17 08:59:59.999,4239.503586,28,0.814655,3552.746817,0


In [470]:
import pandas as pd

# Load data
df_ohlcv = pd.read_csv("C:/Python/Template/Data Science/DATATHON_2025/data/off_chain/ohlcv.csv")
df_funding = pd.read_csv("C:/Python/Template/Data Science/DATATHON_2025/data/off_chain/funding_rate.csv")

# Pastikan timestamp dalam datetime
df_ohlcv['timestamp'] = pd.to_datetime(df_ohlcv['timestamp'])
df_funding['timestamp'] = pd.to_datetime(df_funding['timestamp'])

# Set timestamp sebagai index agar bisa resample atau align
df_ohlcv.set_index('timestamp', inplace=True)
df_funding.set_index('timestamp', inplace=True)

# Gabungkan menggunakan join dan ffill funding_rate
df_merged = df_ohlcv.join(df_funding, how='left')  # gabung berdasarkan timestamp 1 jam
df_merged['fundingRate'] = df_merged['fundingRate'].fillna(method='ffill')  # apply forward fill

# Tetap isi awal data yang belum ada dengan 0.0
df_merged['fundingRate'] = df_merged['fundingRate'].fillna(0.0)

# Tambahkan flag apakah futures sudah aktif
df_merged['is_futures_active'] = (df_merged.index >= pd.Timestamp('2019-09-10')).astype(int)

# Reset index jika perlu simpan ke CSV
df_merged = df_merged.reset_index()
drop = ['ignore']
df_merged = df_merged.drop(drop, axis=1)
df_merged.to_csv("C:/Python/Template/Data Science/DATATHON_2025/data/off_chain/off_chain_btc.csv", index=False)


C:\Users\Elzandi Irfan Zikra\AppData\Local\Temp\ipykernel_22676\415989920.py:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged['fundingRate'] = df_merged['fundingRate'].fillna(method='ffill')  # apply forward fill


In [471]:
df = pd.read_csv("C:/Python/Template/Data Science/DATATHON_2025/data/off_chain/off_chain_btc.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68324 entries, 0 to 68323
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   timestamp            68324 non-null  object 
 1   open                 68324 non-null  float64
 2   high                 68324 non-null  float64
 3   low                  68324 non-null  float64
 4   close                68324 non-null  float64
 5   volume               68324 non-null  float64
 6   close_time           68324 non-null  object 
 7   quote_asset_volume   68324 non-null  float64
 8   num_trades           68324 non-null  int64  
 9   taker_buy_base_vol   68324 non-null  float64
 10  taker_buy_quote_vol  68324 non-null  float64
 11  fundingRate          68324 non-null  float64
 12  is_futures_active    68324 non-null  int64  
dtypes: float64(9), int64(2), object(2)
memory usage: 6.8+ MB


,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,taker_buy_base_vol,taker_buy_quote_vol,fundingRate,is_futures_active
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,2017-08-17 04:59:59.999,202366.138393,171,35.160503,150952.477943,0.0,0
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,2017-08-17 05:59:59.999,100304.823567,102,21.448071,92608.279728,0.0,0
2,2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,2017-08-17 06:59:59.999,31282.312670,36,4.802861,20795.317224,0.0,0
3,2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,2017-08-17 07:59:59.999,19241.058300,25,2.602292,11291.347015,0.0,0
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,2017-08-17 08:59:59.999,4239.503586,28,0.814655,3552.746817,0.0,0


Macro

In [472]:
import os
import pandas as pd
from datetime import timedelta

# ── 1. CONFIGURATION ─────────────────────────────────────────────────────────
INPUT_DIR  = r"C:/Python/Template/Data Science/DATATHON_2025/data/macro/macro1"
OUTPUT_DIR2 = r"C:/Python/Template/Data Science/DATATHON_2025/data/macro"
OUTPUT_DIR = os.path.join(OUTPUT_DIR2, "macro2")
os.makedirs(OUTPUT_DIR, exist_ok=True)

START      = pd.Timestamp("2017-08-17 04:00:00")
END        = pd.Timestamp.now().floor("H")
TIME_INDEX = pd.date_range(start=START, end=END, freq="1H")

# Per-variable settings
VAR_CONFIG = {
    # var: {filename, freq, release_hour, skiprows, timestamp_col, value_col}
    "10y_treasury_yield": {
        "filename":      "10y_treasury_yield.csv",
        "freq":          "daily",
        "release_hour":  0,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     None
    },
    "core_pce": {
        "filename":      "core_pce.csv",
        "freq":          "monthly",
        "release_hour":  14,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     None
    },
    "cpi_us": {
        "filename":      "cpi_us.csv",
        "freq":          "monthly",
        "release_hour":  14,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     None
    },
    "dxy_data": {
        "filename":      "dxy_data.csv",
        "freq":          "daily",
        "release_hour":  0,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     "close"    # pick Close price
    },
    "us_m2": {
        "filename":      "estimated_global_m2.csv",
        "freq":          "monthly",
        "release_hour":  0,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     "us m2"
    },
    "estimated_global_m2": {
        "filename":      "estimated_global_m2.csv",
        "freq":          "monthly",
        "release_hour":  0,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     "estimated global m2"
    },
    "fed_balance_sheet": {
        "filename":      "fed_balance_sheet.csv",
        "freq":          "weekly",
        "release_hour":  22,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     None
    },
    "fed_funds_rate": {
        "filename":      "fed_funds_rate.csv",
        "freq":          "monthly",
        "release_hour":  20,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     None
    },
    "gold_price": {
        "filename":      "gold_price.csv",
        "freq":          "monthly",
        "release_hour":  0,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     None
    },
    "google_trends_bitcoin": {
        "filename":      "google_trends_bitcoin.csv",
        "freq":          "daily",
        "release_hour":  8,
        "skiprows":      True,
        "timestamp_col": None,
        "value_col":     "bitcoin"
    },
    "nasdaq_composite": {
        "filename":      "nasdaq_composite.csv",
        "freq":          "daily",
        "release_hour":  21,
        "skiprows":      None,
        "timestamp_col": None,    # override detection
        "value_col":     "close"    # pick Close price
    },
    "oil_price_brent": {
        "filename":      "oil_price_brent.csv",
        "freq":          "daily",
        "release_hour":  0,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     None
    },
    "oil_price_wti": {
        "filename":      "oil_price_wti.csv",
        "freq":          "daily",
        "release_hour":  0,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     None
    },
    "qqq_etf": {
        "filename":      "qqq_etf.csv",
        "freq":          "daily",
        "release_hour":  21,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     "close"
    },
    "usd_ars": {
        "filename":      "usd_ars.csv",
        "freq":          "daily",
        "release_hour":  0,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     "close"
    },
    "usd_try": {
        "filename":      "usd_try.csv",
        "freq":          "daily",
        "release_hour":  0,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     "close"
    },
    "vix_index": {
        "filename":      "vix_index.csv",
        "freq":          "daily",
        "release_hour":  0,
        "skiprows":      None,
        "timestamp_col": None,
        "value_col":     None
    },
}

# ── 2. RAW READER ─────────────────────────────────────────────────────────────
def read_series(var):
    cfg = VAR_CONFIG[var]
    df = pd.read_csv(
        os.path.join(INPUT_DIR, cfg["filename"]),
        skiprows=[1,2] if cfg["skiprows"] else None
    )
    df.columns = [c.lower().strip() for c in df.columns]
    # timestamp detection / override
    ts_col = cfg["timestamp_col"]
    if not ts_col:
        for opt in ("timestamp","date", "Date", "open_time","unnamed: 0"):
            if opt in df.columns:
                ts_col = opt
                break
    if ts_col not in df.columns:
        raise ValueError(f"{var}: no timestamp column '{ts_col}'")
    df[ts_col] = pd.to_datetime(df[ts_col], errors="coerce", utc=True)\
                  .dt.tz_localize(None)
    df = df.dropna(subset=[ts_col]).set_index(ts_col)
    # pick value column
    val_col = cfg["value_col"]
    if val_col:
        if val_col not in df.columns:
            raise ValueError(f"{var}: missing value_col '{val_col}'")
        s = df[val_col]
    else:
        nums = df.select_dtypes("number").columns
        if not nums.any():
            raise ValueError(f"{var}: no numeric column")
        s = df[nums[0]]
    return s.rename(var)

# ── 3. FILL LOGIC ────────────────────────────────────────────────────────────
def fill_intraday(var):
    s = read_series(var)
    s.index = s.index.floor("H")
    return s.reindex(TIME_INDEX).ffill()

def fill_periodic(var):
    """daily/weekly/monthly: value holds from release until 1h before next."""
    s = read_series(var)
    hr = VAR_CONFIG[var]["release_hour"]
    s.index = pd.DatetimeIndex(s.index).normalize() + timedelta(hours=hr)
    out = pd.Series(index=TIME_INDEX, dtype=float, name=var)
    idxs = list(s.index)
    for i, ts in enumerate(idxs):
        start = ts
        end   = idxs[i+1] - timedelta(hours=1) if i+1 < len(idxs) else END
        out.loc[start:end] = s.loc[ts]
    return out

# ── 4. PROCESS & SAVE ────────────────────────────────────────────────────────
PROCESSORS = {}
for var, cfg in VAR_CONFIG.items():
    func = fill_intraday if cfg["freq"] == "intraday" else fill_periodic
    PROCESSORS[var] = func

if __name__ == "__main__":
    for var, func in PROCESSORS.items():
        print(f"Processing {var}…")
        series = func(var)
        df_out = series.reset_index().rename(columns={"index":"timestamp", var:var})
        df_out.to_csv(os.path.join(OUTPUT_DIR, f"{var}.csv"), index=False)
        print(f" Saved: macro2/{var}.csv ({len(df_out)} rows)")


Processing 10y_treasury_yield…


C:\Users\Elzandi Irfan Zikra\AppData\Local\Temp\ipykernel_22676\2289062523.py:12: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  END        = pd.Timestamp.now().floor("H")
C:\Users\Elzandi Irfan Zikra\AppData\Local\Temp\ipykernel_22676\2289062523.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  TIME_INDEX = pd.date_range(start=START, end=END, freq="1H")


 Saved: macro2/10y_treasury_yield.csv (68485 rows)
Processing core_pce…
 Saved: macro2/core_pce.csv (68485 rows)
Processing cpi_us…
 Saved: macro2/cpi_us.csv (68485 rows)
Processing dxy_data…
 Saved: macro2/dxy_data.csv (68485 rows)
Processing us_m2…
 Saved: macro2/us_m2.csv (68485 rows)
Processing estimated_global_m2…
 Saved: macro2/estimated_global_m2.csv (68485 rows)
Processing fed_balance_sheet…
 Saved: macro2/fed_balance_sheet.csv (68485 rows)
Processing fed_funds_rate…
 Saved: macro2/fed_funds_rate.csv (68485 rows)
Processing gold_price…
 Saved: macro2/gold_price.csv (68485 rows)
Processing google_trends_bitcoin…
 Saved: macro2/google_trends_bitcoin.csv (68485 rows)
Processing nasdaq_composite…
 Saved: macro2/nasdaq_composite.csv (68485 rows)
Processing oil_price_brent…
 Saved: macro2/oil_price_brent.csv (68485 rows)
Processing oil_price_wti…
 Saved: macro2/oil_price_wti.csv (68485 rows)
Processing qqq_etf…
 Saved: macro2/qqq_etf.csv (68485 rows)
Processing usd_ars…
 Saved: macro

In [473]:
import os
import pandas as pd

# === Konfigurasi Path ===
folder_path = r"C:/Python/Template/Data Science/DATATHON_2025/data/macro/macro2"
file_list = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

# === Inisialisasi DataFrame Gabungan ===
df_merged = None

# === Gabungkan Semua File Berdasarkan Kolom 'timestamp' ===
for filename in file_list:
    full_path = os.path.join(folder_path, filename)
    df = pd.read_csv(full_path)

    # Normalisasi nama kolom dan cari kolom waktu
    df.columns = [col.lower().strip() for col in df.columns]
    if 'timestamp' not in df.columns:
        print(f"⚠️ Skip {filename} — kolom 'timestamp' tidak ditemukan.")
        continue

    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df = df.dropna(subset=['timestamp'])

    # Ambil kolom numerik pertama sebagai nilai
    value_col = df.select_dtypes(include=['float64', 'int64']).columns[0]
    varname = filename.replace(".csv", "").lower()
    df = df[['timestamp', value_col]].rename(columns={value_col: varname})

    # Gabungkan
    if df_merged is None:
        df_merged = df
    else:
        df_merged = pd.merge(df_merged, df, on='timestamp', how='outer')

# === Simpan Hasil Gabungan ===
folder_path2 = r"C:/Python/Template/Data Science/DATATHON_2025/data/macro/macro3"
output_path1 = os.path.join(folder_path2, "macro_btc.csv")
output_path2 = os.path.join(folder_path2, "macro_btc.xlsx")
df_merged.to_csv(output_path1, index=False)
df_merged.to_excel(output_path2, index=False)

print(f"✅ Berhasil digabung dan disimpan di: {output_path1}")


✅ Berhasil digabung dan disimpan di: C:/Python/Template/Data Science/DATATHON_2025/data/macro/macro3\macro_btc.csv


In [474]:
df = pd.read_csv("C:/Python/Template/Data Science/DATATHON_2025/data/macro/macro3\macro_btc.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68485 entries, 0 to 68484
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   timestamp              68485 non-null  object 
 1   10y_treasury_yield     65893 non-null  float64
 2   core_pce               68485 non-null  float64
 3   cpi_us                 68485 non-null  float64
 4   dxy_data               68485 non-null  float64
 5   estimated_global_m2    68485 non-null  float64
 6   fed_balance_sheet      68485 non-null  float64
 7   fed_funds_rate         68485 non-null  float64
 8   gold_price             68485 non-null  float64
 9   google_trends_bitcoin  68485 non-null  float64
 10  nasdaq_composite       68485 non-null  float64
 11  oil_price_brent        66517 non-null  float64
 12  oil_price_wti          65629 non-null  float64
 13  qqq_etf                68485 non-null  float64
 14  usd_ars                68485 non-null  float64
 15  us

,timestamp,10y_treasury_yield,core_pce,cpi_us,dxy_data,estimated_global_m2,fed_balance_sheet,fed_funds_rate,gold_price,google_trends_bitcoin,nasdaq_composite,oil_price_brent,oil_price_wti,qqq_etf,usd_ars,usd_try,us_m2,vix_index
0,2017-08-17 04:00:00,2.19,100.118,245.183,93.620003,13674.6,4462871.0,1.16,47.9,84.0,6345.109863,50.37,47.07,136.829102,17.200001,3.51604,13674.6,15.55
1,2017-08-17 05:00:00,2.19,100.118,245.183,93.620003,13674.6,4462871.0,1.16,47.9,84.0,6345.109863,50.37,47.07,136.829102,17.200001,3.51604,13674.6,15.55
2,2017-08-17 06:00:00,2.19,100.118,245.183,93.620003,13674.6,4462871.0,1.16,47.9,84.0,6345.109863,50.37,47.07,136.829102,17.200001,3.51604,13674.6,15.55
3,2017-08-17 07:00:00,2.19,100.118,245.183,93.620003,13674.6,4462871.0,1.16,47.9,84.0,6345.109863,50.37,47.07,136.829102,17.200001,3.51604,13674.6,15.55
4,2017-08-17 08:00:00,2.19,100.118,245.183,93.620003,13674.6,4462871.0,1.16,47.9,78.0,6345.109863,50.37,47.07,136.829102,17.200001,3.51604,13674.6,15.55


In [3]:
import pandas as pd
import os

# 1. Path input & output
input_path  = "C:/Python/Template/Data Science/DATATHON_2025/data/macro/macro3/macro_btc.csv"
output_path = os.path.join(
    os.path.dirname(input_path),
    "macro_btc2.csv"
)

# 2. Load data, parse timestamp
df = pd.read_csv(input_path, parse_dates=["timestamp"])

# 3. Set timestamp jadi index dan pastikan terurut
df.set_index("timestamp", inplace=True)
df.sort_index(inplace=True)

# 4. Imputasi missing dengan forward‐fill
#    - Memakai nilai terakhir yang ada, termasuk untuk gap weekend
df_imputed = df.ffill()

# 5. (Optional) Jika ada missing di awal file, fill dengan first valid
df_imputed = df_imputed.bfill()

# 6. Cek apakah masih ada missing
print("Missing setelah fill per kolom:")
print(df_imputed.isna().sum())

# 7. Simpan ke CSV baru
df_imputed.to_csv(output_path, index=True)

print(f"✅ File hasil imputasi disimpan di:\n   {output_path}")


Missing setelah fill per kolom:
10y_treasury_yield       0
core_pce                 0
cpi_us                   0
dxy_data                 0
estimated_global_m2      0
fed_balance_sheet        0
fed_funds_rate           0
gold_price               0
google_trends_bitcoin    0
nasdaq_composite         0
oil_price_brent          0
oil_price_wti            0
qqq_etf                  0
usd_ars                  0
usd_try                  0
us_m2                    0
vix_index                0
dtype: int64
✅ File hasil imputasi disimpan di:
   C:/Python/Template/Data Science/DATATHON_2025/data/macro/macro3\macro_btc2.csv


In [4]:
df = pd.read_csv("C:/Python/Template/Data Science/DATATHON_2025/data/macro/macro3\macro_btc2.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68485 entries, 0 to 68484
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   timestamp              68485 non-null  object 
 1   10y_treasury_yield     68485 non-null  float64
 2   core_pce               68485 non-null  float64
 3   cpi_us                 68485 non-null  float64
 4   dxy_data               68485 non-null  float64
 5   estimated_global_m2    68485 non-null  float64
 6   fed_balance_sheet      68485 non-null  float64
 7   fed_funds_rate         68485 non-null  float64
 8   gold_price             68485 non-null  float64
 9   google_trends_bitcoin  68485 non-null  float64
 10  nasdaq_composite       68485 non-null  float64
 11  oil_price_brent        68485 non-null  float64
 12  oil_price_wti          68485 non-null  float64
 13  qqq_etf                68485 non-null  float64
 14  usd_ars                68485 non-null  float64
 15  us

,timestamp,10y_treasury_yield,core_pce,cpi_us,dxy_data,estimated_global_m2,fed_balance_sheet,fed_funds_rate,gold_price,google_trends_bitcoin,nasdaq_composite,oil_price_brent,oil_price_wti,qqq_etf,usd_ars,usd_try,us_m2,vix_index
0,2017-08-17 04:00:00,2.19,100.118,245.183,93.620003,13674.6,4462871.0,1.16,47.9,84.0,6345.109863,50.37,47.07,136.829102,17.200001,3.51604,13674.6,15.55
1,2017-08-17 05:00:00,2.19,100.118,245.183,93.620003,13674.6,4462871.0,1.16,47.9,84.0,6345.109863,50.37,47.07,136.829102,17.200001,3.51604,13674.6,15.55
2,2017-08-17 06:00:00,2.19,100.118,245.183,93.620003,13674.6,4462871.0,1.16,47.9,84.0,6345.109863,50.37,47.07,136.829102,17.200001,3.51604,13674.6,15.55
3,2017-08-17 07:00:00,2.19,100.118,245.183,93.620003,13674.6,4462871.0,1.16,47.9,84.0,6345.109863,50.37,47.07,136.829102,17.200001,3.51604,13674.6,15.55
4,2017-08-17 08:00:00,2.19,100.118,245.183,93.620003,13674.6,4462871.0,1.16,47.9,78.0,6345.109863,50.37,47.07,136.829102,17.200001,3.51604,13674.6,15.55
